In [2]:
import time, datetime, re
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
start_time=time.time()
print (datetime.datetime.now())
def compareTwoString(strA, strB):
    result=""
    junkLetters="""\s□abcdefghijklmnopqrstuvwxyzABCDEFGHIJKL
    MNOPQRSTUVWXYZ0123456789<>:'`;.?"=-,\/《》「」『』（），、。．：？！－…；·：【】○"""
    punc ="《》「」『』（），、。．：？！－…；·：【】○"
    puncReg =re.compile(r"([《》「」『』（），、。．：？！－…；·：【】○]+)")
    charReg =re.compile(r"([\u4E00-\u9FFF]+)")
    start_i=0
    s = SequenceMatcher(lambda x: x in junkLetters, strA, strB, autojunk=False)
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        repStr = strB[j1:j2].strip()
        if (tag=="insert"):
            match=re.search(charReg, repStr)
            if match:
                if repStr[0] in punc:
                    result += repStr[0]+"<seg n='insert'>"+repStr[1:]+"</seg>"
                else:
                    result += "<seg n='insert'>"+repStr+"</seg>"
            else:
                result += repStr
        elif (tag=="replace") and len(repStr) > 0:
            repStrGroup=re.findall(r"[《》「」『』，、。．：？！－…；·：【】○]+|[\u4E00-\u9FFF]+", repStr)
            if len(repStrGroup)>3:
                repStrHan=re.findall(r"[\u4E00-\u9FFF]+", repStr)
                if len("".join(repStrHan))==i2-i1:
                    start_i=i1
                    group=0
                    for str in repStrGroup:
                        if str[0] in punc:
                            result += str
                        else:
                            result += strA[start_i:start_i+len(repStrGroup[group])]
                            start_i += len(repStrGroup[group])
                        group += 1
                else:
                    result += strA[i1:i2]+"<seg n='replace'>"+repStr\
                    +"</seg>"
            elif len(repStrGroup)==3:
                if repStrGroup[0][0] in punc:
                    result += repStrGroup[0]+strA[i1:i2]\
                    +repStrGroup[2]
                else:
                    try: 
                        result += strA[i1:i1+1]+repStrGroup[1]\
                        +strA[i1+1:i2]
                    except:
                        result += strA[i1:i2]+"<seg n='replace'>"\
                        +repStr+"</seg>"
            elif len(repStrGroup)==2:
                if repStrGroup[0][0] in punc and \
                repStrGroup[1][0] in punc:
                    result += repStrGroup[0]+strA[i1:i2]\
                    +repStrGroup[1]
                elif repStrGroup[0][0] in punc:
                    result += repStrGroup[0]+strA[i1:i2]
                elif repStrGroup[1][0] in punc:
                    result += strA[i1:i2]+repStrGroup[1]
                else:
                    result += strA[i1:i2]
            elif len(repStrGroup)==1:
                repStr=repStrGroup[0]
                if repStr[0] in punc:
                    match = re.search("[「《〈]", repStr)
                    if match:
                        result += repStr[:match.start(0)]+\
                        strA[i1:i2]+repStr[match.start(0):]
                    else:
                        result += repStr+strA[i1:i2]
                else:
                    result += strA[i1:i2]
            else:
                result += strA[i1:i2]
        else:
            result += strA[i1:i2]
    return result
finalResult="""<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml"
	schematypens="http://purl.oclc.org/dsdl/schematron"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>毛诗正义</title>
                <author><date>汉</date><persName>孔安国</persName>传</author>
                <author><date>唐</date><persName>孔颖达</persName>疏</author>
            </titleStmt>
            <publicationStmt>
                <p>Publication Information</p>
            </publicationStmt>
            <sourceDesc>
                <p>ishare</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text>
        <body>

"""
with open(r"G:\小組雲端硬碟\儒家經典注疏\十三經注疏\漢子電子文獻-刪校勘記\左傳.xml",
          "r", encoding="utf8") as fileA:
    soupA = BeautifulSoup(fileA.read(), "xml")
with open(r"G:\我的雲端硬碟\GitHub\kanripo-data\data\KR1e\KR1e1000.xml",
         "r", encoding="utf8") as fileB:
    soupB = BeautifulSoup(fileB.read(), "xml")
listA=[]
listB=[]
for divA in soupA.body.find_all("div",recursive=False):
    listA.append(divA.prettify())
for divB in soupB.body.find_all("div",recursive=False):
    listB.append(divB.prettify())
for i in range(len(listA)):
    strA = listA[i]
    strB = listB[i]
    finalResult += compareTwoString(strA, strB)
    print(i)
finalResult += "</body></text></TEI>"
with open(r"d:\temp\tempResult.xml", "w", encoding="utf8") as infile:
    infile.write(finalResult)
print (datetime.datetime.now())
print(time.time()-start_time)

2018-07-11 12:03:03.877269
0
1
2
3
4
5
6
7
8
9
10
11
12
2018-07-11 12:11:06.705590
482.828320980072


In [ ]:
以下試圖克服insert和replace過長字串的問題。

In [ ]:
import time, datetime, re
from bs4 import BeautifulSoup
from difflib import SequenceMatcher
start_time=time.time()
print (datetime.datetime.now())
global junkLetters
junkLetters = """\s□abcdefghijklmnopqrstuvwxyzABCDEFGHIJKL
    MNOPQRSTUVWXYZ0123456789<>:'`;.?"=-,\/《》「」『』（），、。．：？！－…；·：【】○"""
def compareTwoString(strA, strB):
    result=""
    extraInsert=False
    def insertPunc(strX, c):
        if c in "《「『":
            strX=strX+c
        else:
            for i in range(len(strX)):
                if strX[-(i+1)] in junkLetters:
                    pass
                else:
                    strX = strX[:-(i+1)]+c+strX[-(i+1):]
        return strX
    punc ="《》「」『』（），、。．：？！－…；·：【】○"
    puncReg =re.compile(r"[\s《》「」『』（），、。．：？！－…；·：【】○]+")
    charReg =re.compile(r"[\u4E00-\u9FFF]+")
    charReg_s =re.compile(r"[\s\u4E00-\u9FFF]+")
    puncReg_charReg = re.compile(r"[\s《》「」『』（），、。．：？！－…；·：【】○]+|[\s\u4E00-\u9FFF]+")
    start_i=0
    s = SequenceMatcher(lambda x: x in junkLetters, strA, strB, autojunk=False)
    for tag, i1, i2, j1, j2 in s.get_opcodes():
        repStr = strB[j1:j2].strip()
        if (tag=="insert"):
            match=puncReg_charReg.findall(repStr)
            if match[0][0]in punc:
                for c in match[0]:
                    result = insertPunc(result, c)
            if len(match)==1:
                if match[0][0]in punc:
                    pass
                else:
                    result += "<seg n='insert'>"+match[0]+"</seg>"
            elif len(match)==2:
                if match[0][0]in punc:
                    result += match[1]
                else:
                    result += "<seg n='insert'>"+match[0]+"</seg>"+match[1]
            else:
                extraInsert = True
                if match[0][0]in punc:
                    extraInsertStr="".join(match[1:])
                else:
                    extraInsertStr="".join(match)
                extraInsertLen=len(extraInsertStr)
                tempContainer=""
        else:
            if extraInsert:
                tempContainer += strA[i1:i2]
                if len(tempContainer) < extraInsertLen:
                    pass
                else:
                    t = SequenceMatcher(lambda x: x=="\s", tempContainer, extraInsertStr, autojunk=False)
                    for tag2, i21, i22, j21, j22 in t.get_opcodes():
                        tempStr=extraInsertStr[j2i:j22]
                        if tag2=="insert":
                            result += charReg.sub("<seg n='insert'>\g<0></seg>",tempStr)
                        else:
                            result += tempContainer[i21:i22]
                    extraInsert=False
            else:
                if (tag=="replace") and len(repStr) > 0:
                    repStrGroup=re.findall(r"[《》「」『』，、。．：？！－…；·：【】○]+|[\u4E00-\u9FFF]+", repStr)
                    if len(repStrGroup)>3:
                        extraInsert = True
                        if repStrGroup[0][0]in punc:
                            for c in repStrGroup[0]:
                                result=insertPunc(result, c)
                            extraInsertStr="".join(repStrGroup[1:])
                        else:
                            extraInsertStr="".join(repStrGroup)
                        extraInsertLen=len(extraInsertStr)
                        tempContainer=""
                    elif len(repStrGroup)==3:
                        if repStrGroup[0][0] in punc:
                            result += repStrGroup[0]+strA[i1:i2]\
                            +repStrGroup[2]
                        else:
                            try: 
                                result += strA[i1:i1+1]+repStrGroup[1]\
                                +strA[i1+1:i2]
                            except:
                                result += strA[i1:i2]+"<seg n='replace'>"\
                                +repStr+"</seg>"
                    elif len(repStrGroup)==2:
                        if repStrGroup[0][0] in punc and \
                        repStrGroup[1][0] in punc:
                            result += repStrGroup[0]+strA[i1:i2]\
                            +repStrGroup[1]
                        elif repStrGroup[0][0] in punc:
                            result += repStrGroup[0]+strA[i1:i2]
                        elif repStrGroup[1][0] in punc:
                            result += strA[i1:i2]+repStrGroup[1]
                        else:
                            result += strA[i1:i2]
                    elif len(repStrGroup)==1:
                        repStr=repStrGroup[0]
                        if repStr[0] in punc:
                            match = re.search("[「《〈]", repStr)
                            if match:
                                result += repStr[:match.start(0)]+\
                                strA[i1:i2]+repStr[match.start(0):]
                            else:
                                result += repStr+strA[i1:i2]
                        else:
                            result += strA[i1:i2]
                    else:
                        result += strA[i1:i2]
                else:
                    result += strA[i1:i2]
    return result
finalResult="""<?xml version="1.0" encoding="UTF-8"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>
<?xml-model href="http://www.tei-c.org/release/xml/tei/custom/schema/relaxng/tei_all.rng" type="application/xml"
	schematypens="http://purl.oclc.org/dsdl/schematron"?>
<TEI xmlns="http://www.tei-c.org/ns/1.0">
    <teiHeader>
        <fileDesc>
            <titleStmt>
                <title>毛诗正义</title>
                <author><date>汉</date><persName>孔安国</persName>传</author>
                <author><date>唐</date><persName>孔颖达</persName>疏</author>
            </titleStmt>
            <publicationStmt>
                <p>Publication Information</p>
            </publicationStmt>
            <sourceDesc>
                <p>ishare</p>
            </sourceDesc>
        </fileDesc>
    </teiHeader>
    <text>
        <body>

"""
with open(r"G:\小組雲端硬碟\儒家經典注疏\十三經注疏\漢子電子文獻-刪校勘記\左傳.xml",
          "r", encoding="utf8") as fileA:
    soupA = BeautifulSoup(fileA.read(), "xml")
with open(r"G:\我的雲端硬碟\GitHub\kanripo-data\data\KR1e\KR1e1000.xml",
         "r", encoding="utf8") as fileB:
    soupB = BeautifulSoup(fileB.read(), "xml")
listA=[]
listB=[]
for divA in soupA.body.find_all("div",recursive=False):
    listA.append(divA.prettify())
for divB in soupB.body.find_all("div",recursive=False):
    tempString=""
    for string in divB.strings:
        tempString += str(string)
    listB.append(tempString)
for i in range(len(listA)):
    strA = listA[i]
    strB = listB[i]
    finalResult += compareTwoString(strA, strB)
    print(i)
finalResult += "</body></text></TEI>"
with open(r"d:\temp\tempResult.xml", "w", encoding="utf8") as infile:
    infile.write(finalResult)
print (datetime.datetime.now())
print(time.time()-start_time)